## Experiment 2: Using NLI to obtain constraints between WDIK facts and raw answers, and solving them using Z3 to flip answers if required

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import experiment_setup

In [18]:
from belief.evaluation import load_facts
from belief.nli import load_nli_model, load_nli_tokenizer, run_nli
from belief.lmbb import Proposition
import json
import random
from z3 import *

In [7]:
facts = load_facts('data/calibration_facts.json', num_batches=1)[0]

with open('cache/wdik.json', 'r') as f:
    wdik = json.load(f)

with open('data/constraints_v2.json', 'r') as f:
    constraint_data = json.load(f)
    
with open('cache/raw_outs_calib.json', 'r') as f:
    raw_outs = json.load(f)['outs']

In [19]:
raw_out_beliefs = {}
for d in raw_outs:
    boolean = max([(d['yes'], True), (d['no'], False)], key=lambda t : t[0])[1]
    prop = Proposition.from_sent(d['prop'], boolean=boolean)
    raw_out_beliefs[prop.sentence] = prop

In [11]:
NUM_FACTS = 3

In [21]:
fact = random.choice(facts)
print(fact)
print()

print(raw_out_beliefs[fact.sentence])

wdik_facts = random.sample(wdik[fact.subject], NUM_FACTS)
print(wdik_facts)
print()


(adder,IsA,vegetable, False, -99999.0)

(adder,IsA,vegetable, True, -99999.0)
['An adder is a device for adding volume to liquids.', 'An adder has the properties of shape, size, weight.', 'An adder is capable of adding, removing, combining.']

